# Data preprocessing TO DO: filter op variabelen die wij interessant vinden.

## Datasets kiezen
Wij hebben drie datasets gekozen om te vergelijken. Als eerste hebben wij de dataset ["Police-recorded offences by offence category"](https://ec.europa.eu/eurostat/databrowser/view/crim_off_cat/default/table?lang=en&category=crim.crim_off) van Eurostat om verschillende categorieën criminaliteit in de deelnemende landen te kunnen onderzoeken. Wij hebben geen dataset gevonden die meer landen verkende met eenzelfde diepgang, vandaar dat wij ons hebben toegespitst op alleen de landen die in deze dataset voor komen.

Wij hebben ook de dataset ["Economy & Growth"](https://data.worldbank.org/topic/economy-and-growth) van World Bank Group gebruikt. Deze dataset bevat verschillende economische statistieken van een groot aantal landen over een lange periode. Deze gebruikten wij om correlaties te identificeren tussen de categorieën criminaliteit uit de Eurostat datbase, en verschillende economische factoren en -trends.

Ten slotte hebben wij de dataset ["World Bank, Poverty and Inequality Platform. (Gini Index)"](https://databank.worldbank.org/reports.aspx?source=2&series=SI.POV.GINI&country=), ook van World bank group, gebruikt als een (versimpelde) weergave van de welvaartsongelijkheid in deelnemende landen.



## Preprocessing
Het data preprocessen gebeurde in een aantal stappen. Wij hebben hier python code gebruikt met de pandas library.

1. De dataset "Police-recorded offences by offence category" van de EU hebben wij omgevormd zodat iedere soort misdaad een eigen kolom kreeg.
2. Wij hebben de andere twee dataset, "Economy & Growth" en "World Bank, Poverty and Inequality Platform. (Gini Index)" omgevormd van 'wide' naar 'long-format'.
3. Wij hebben deze laatste twee databases gefilterd op jaar en land, zodat alleen de jaren en landen die voor komen in de EU database over blijven. Op deze manier kunnen we voor de EU landen correlaties tussen economische variabelen en verschillende soorten criminaliteit identificeren.

## Variabelen
Beschrijving van de gebruikte variabelen

Lorem ipsum si dolor amet

## Gebruikte code

In [ ]:
import pandas as pd
######################################################
# Converting wide-format datasets to long-format ones#
######################################################

def convert_wide_to_long(file_path, skiprows=4, id_vars=None, var_name="Year", value_name="Value"):
    # Load the CSV file
    df = pd.read_csv(file_path, skiprows=skiprows)
    
    # Automatically determine id_vars if not provided
    if id_vars is None:
        id_vars = df.columns[:4]  # First 4 columns are IDs
    
    # Melt the DataFrame
    df_long = pd.melt(df, id_vars=id_vars, var_name=var_name, value_name=value_name)
    
    # Optionally drop rows with missing values in the value column
    # df_long.dropna(subset=[value_name], inplace=True)

    return df_long

# Convert economy csv to long format
file_path1 = "og_csv_files/API_3_DS2_en_csv_v2_390106.csv"
world_bank_long_format = convert_wide_to_long(file_path1)

# convert GINI csv to long format
filepath2 = "og_csv_files/API_SI.POV.GINI_DS2_en_csv_v2_2566.csv"
gini_long_format = convert_wide_to_long(filepath2)

#save as new files to [path]
#output_csv_path1 = "world bank long format2.csv"
#output_csv_path2 = "gini long2.csv"
#df_long1.to_csv(output_csv_path1, index=False)
#df_long2.to_csv(output_csv_path2, index=False)

###############################################################################
# EU dataset pivot so that the different types of crime get their own columns #
###############################################################################

# Load the dataset
eu_df = pd.read_csv("/home/jason/huiswerk/Semester 2/information visualisation/datastory/InfoVis_Datastory/docs/og_csv_files/estat_crim_off_cat_filtered_en.csv")

def convert_and_pivot(df, preferred_unit):
    # Validate unit
    if preferred_unit not in ['Number', 'Per hundred thousand inhabitants']:
        raise ValueError("Preferred unit must be 'Number' or 'Per hundred thousand inhabitants'.")

    # Filter based on unit of measure
    filtered_df = df[df['Unit of measure'] == preferred_unit].copy()

    # Pivot so each ICCS crime type becomes a column
    pivot_df = filtered_df.pivot_table(
        index=['Geopolitical entity (reporting)','geo', 'TIME_PERIOD'],
        columns='International classification of crime for statistical purposes (ICCS)',
        values='OBS_VALUE'
    ).reset_index()

    return pivot_df

# usage
eu_converted_per_100k = convert_and_pivot(eu_df, 'Per hundred thousand inhabitants')
eu_converted_absolute = convert_and_pivot(eu_df, 'Number')

# Save the result
output_path_per_100k = 'europe_crime_definitive_per_100k.csv'
eu_converted_per_100k.to_csv(output_path_per_100k, index=False)
print(f"Filtered and pivoted data saved to {output_path_per_100k}")

output_path_absolute = 'europe_crime_definitive_absolute.csv'
eu_converted_absolute.to_csv(output_path_absolute, index=False)
print(f"Filtered and pivoted data saved to {output_path_absolute}")





Filtered and pivoted data saved to europe_crime_definitive_per_100k.csv
Filtered and pivoted data saved to europe_crime_definitive_absolute.csv


In [4]:
import pandas as pd

########################################################################
#Sort economy and gini to only use countries also present in eu dataset#
########################################################################

def filter_to_european_countries(crime_file, gini_file, world_bank_file):
    """
    Filters the gini and world bank datasets to include only countries present in the european crime dataset.

    Parameters:
        crime_file (str): Path to 'europe_crime_definitive.csv'
        gini_file (pd.df): Gini file in long format
        world_bank_file (pd.df): World bank file in long format

    Returns:
        Tuple of filtered DataFrames: (filtered_gini_df, filtered_world_bank_df)
    """
    # Load the crime dataset
    crime_df = pd.read_csv(crime_file)
    
    # Extract unique country names
    european_countries = crime_df['Geopolitical entity (reporting)'].unique()
    
    # Load GINI and World Bank datasets
    gini_df = gini_file
    world_bank_df = world_bank_file
    
    # Filter GINI and World Bank datasets to only include European countries
    filtered_gini_df = gini_df[gini_df['Country Name'].isin(european_countries)].copy()
    filtered_world_bank_df = world_bank_df[world_bank_df['Country Name'].isin(european_countries)].copy()
    
    return filtered_gini_df, filtered_world_bank_df

gini_filtered, world_bank_filtered = filter_to_european_countries('europe_crime_definitive.csv', gini_long_format, world_bank_long_format)

FileNotFoundError: [Errno 2] No such file or directory: 'europe_crime_definitive.csv'

In [ ]:
#########################################################
#Filter by the timespan of the EU database, 2008 onwards#
#########################################################
def filter_years_from_2008(gini_df, world_bank_df, start_year=2008):
    """
    Filters the GINI and World Bank DataFrames to include only data from start_year onward.

    Parameters:
        gini_df (pd.DataFrame): Filtered GINI DataFrame.
        world_bank_df (pd.DataFrame): Filtered World Bank DataFrame.
        start_year (int): The earliest year to include (default is 2008).

    Returns:
        Tuple of DataFrames filtered by year: (gini_filtered_by_year, world_bank_filtered_by_year)
    """
    # Make the string entries of the year column be interpreted as numbers
    gini_df['Year'] = pd.to_numeric(gini_df['Year'], errors='coerce')
    world_bank_df['Year'] = pd.to_numeric(world_bank_df['Year'], errors='coerce')

    # Filter out all data with a year not 2008 or onwards
    gini_filtered_by_year = gini_df[gini_df['Year'] >= start_year].copy()
    world_bank_filtered_by_year = world_bank_df[world_bank_df['Year'] >= start_year].copy()
    
    return gini_filtered_by_year, world_bank_filtered_by_year

gini_definitive, world_bank_definitive = filter_years_from_2008(gini_filtered, world_bank_filtered)

# Save the final results
gini_definitive.to_csv('gini_definitive.csv')
world_bank_definitive.to_csv('world_bank_definitive.csv')
